## Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path
from typing import List

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path    
    status_file: Path
    all_required_files: List


## Configuration Manager

In [6]:
from textSummarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from textSummarizer.utils.common import create_directories, read_yaml



class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation

        create_directories([self.config.artifacts_root])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            status_file=config.status_file,
            all_required_files=config.all_required_files
        )
        return data_validation_config

## Component

In [7]:
import os
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size
from pathlib import Path


class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_files_exist(self)-> bool:
        try:
            validation_status = None

            all_files = os.listdir(os.path.join("artifacts","data_ingestion","samsum_dataset"))

            for file in all_files:
                if file not in self.config.all_required_files:
                    validation_status = False
                    with open(self.config.status_file, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.status_file, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

    def initiate_data_validation(self):
        return self.validate_all_files_exist()
       

## Pipeline

In [11]:

from textSummarizer.logging import logger


config = ConfigurationManager()
data_validation_config = config.get_data_validation_config()
data_validation = DataValidation(config=data_validation_config)
data_validation.initiate_data_validation()


[2023-10-11 12:24:13,516:  textSummarizerLogger: INFO: common: Line 28 - yaml file: config/config.yaml loaded successfully]
[2023-10-11 12:24:13,519:  textSummarizerLogger: INFO: common: Line 28 - yaml file: params.yaml loaded successfully]
[2023-10-11 12:24:13,520:  textSummarizerLogger: INFO: common: Line 46 - created directory at: artifacts]
[2023-10-11 12:24:13,521:  textSummarizerLogger: INFO: common: Line 46 - created directory at: artifacts]


FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/data_validation/status.txt'

## Main

In [7]:
from textSummarizer.pipeline.stage_01_data_ingestion import DataIngestionTrainingPipeline
from textSummarizer.logging import logger

STAGE_NAME = "Data Ingestion stage"
try:
    logger.info(f">>>>>> stage {STAGE_NAME} started <<<<<<")
    data_ingestion = DataIngestionTrainingPipeline()
    data_ingestion.main()
    logger.info(f">>>>>> stage {STAGE_NAME} completed <<<<<<\n\nx==========x")
except Exception as e:
    logger.exception(e)
    raise e


[2023-10-10 10:33:33,326:  textSummarizerLogger: INFO: 367653918: Line 6 - >>>>>> stage Data Ingestion stage started <<<<<<]
[2023-10-10 10:33:33,328:  textSummarizerLogger: INFO: common: Line 28 - yaml file: config/config.yaml loaded successfully]
[2023-10-10 10:33:33,330:  textSummarizerLogger: INFO: common: Line 28 - yaml file: params.yaml loaded successfully]
[2023-10-10 10:33:33,330:  textSummarizerLogger: INFO: common: Line 46 - created directory at: artifacts]
[2023-10-10 10:33:33,330:  textSummarizerLogger: INFO: common: Line 46 - created directory at: artifacts]
[2023-10-10 10:33:33,331:  textSummarizerLogger: INFO: data_ingestion: Line 22 - File already exists of size: ~ 8 MB]
[2023-10-10 10:33:33,410:  textSummarizerLogger: INFO: 367653918: Line 9 - >>>>>> stage Data Ingestion stage completed <<<<<<

x==========x]
